In [4]:
from ultralytics import YOLO

model = YOLO("yolo11x-pose.pt")

model.to('cuda')

import os 
# loop through all the files in the folder
results = []
for filename in os.listdir('images'):
    print(filename)
    results = model.predict("images/"+filename, imgsz=320, conf=0.5) + results


#results = model.predict("images/hammer curl",save= True, imgsz=320, conf=0.5, device = 'gpu')





bench press


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1595 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\bench press_1000001.jpg: 256x320 1 person, 82.9ms
image 2/1595 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\bench press_100001.jpg: 256x320 2 persons, 29.3ms
image 3/1595 c:\Users\Samet\Desktop\3. Grade 1. Term\AIN 311 - 313\project\yolo model\images\bench press\bench press_1000011.jpg: 256x32

In [5]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # Optional: Display GPU name

True


AssertionError: Invalid device id

In [ ]:
import torch
import math
import csv

# Joint labels for reference
joint_labels = ["Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear",
                "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow",
                "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
                "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"]

# Function to calculate angle between three points (p1, p2, p3)
def calculate_angle(p1, p2, p3):
    # Create vectors
    v1 = p1 - p2
    v2 = p3 - p2
    
    # Calculate dot product and magnitudes
    dot_product = torch.dot(v1, v2)
    magnitude_v1 = torch.norm(v1)
    magnitude_v2 = torch.norm(v2)
    
    # Prevent division by zero
    if magnitude_v1 == 0 or magnitude_v2 == 0:
        return None  # Undefined angle
    
    # Calculate angle in radians
    angle_rad = torch.acos(dot_product / (magnitude_v1 * magnitude_v2))
    # Convert to degrees
    return math.degrees(angle_rad.item())

# Function to calculate Euclidean distance between two points (p1, p2)
def calculate_distance(p1, p2):
    return torch.norm(p1 - p2).item()

# List to store calculated angles and distances for each image
angles_per_image = []

# Define joint triplets for angle calculations
joint_triplets = [
    (5, 7, 9),  # Left Elbow
    (6, 8, 10), # Right Elbow
    (5, 11, 13),# Left Hip
    (6, 12, 14),# Right Hip
    (7, 5, 11), # Left Shoulder
    (8, 6, 12), # Right Shoulder
    (11, 13, 15),# Left Knee
    (12, 14, 16) # Right Knee
]

# Iterate over each image in the batch
for img_idx in range(len(results)):  # Iterate over images (batch size)
    if len(results[img_idx]) == 0:
        print(f"No keypoints detected in image {img_idx + 1}")
        continue
    if len(results[img_idx]) > 1:
        print(f"Multiple detections in image {img_idx + 1}")
        continue
    image_angles = {"Image": img_idx + 1}
    
    # Calculate angles and distances for each joint triplet
    for triplet in joint_triplets:
        p1 = results[img_idx].keypoints.xy[0][triplet[0]]  # Correctly access the joint coordinates
        p2 = results[img_idx].keypoints.xy[0][triplet[1]]
        p3 = results[img_idx].keypoints.xy[0][triplet[2]]
        
        # Make sure the coordinates are tensors and have the shape [2] (x, y)
        angle = calculate_angle(p1, p2, p3)
        image_angles[f"{joint_labels[triplet[0]]}-{joint_labels[triplet[1]]}-{joint_labels[triplet[2]]}"] = angle
    
    # Calculate and add distances for each joint pair
    for i in range(len(results[img_idx].keypoints.xy[0])):
        for j in range(i + 1, len(results[img_idx].keypoints.xy[0])):
            p1 = results[img_idx].keypoints.xy[0][i]
            p2 = results[img_idx].keypoints.xy[0][j]
            distance = calculate_distance(p1, p2)
            image_angles[f"{joint_labels[i]}-{joint_labels[j]}_distance"] = distance

    #add the positions of the joints
    for i in range(len(results[img_idx].keypoints.xy[0])):
        p1 = results[img_idx].keypoints.xy[0][i]
        image_angles[f"{joint_labels[i]}_position_y"] = p1[1]

        # to detect the position of the joint in the image

    

    
    
    # Add the "Label" field to each image
    image_angles["Label"] = results[img_idx].path.split("\\")[-2]
    
    angles_per_image.append(image_angles)

# Save the angles and distances for all images as a CSV file
csv_file_path = "joint_angles_and_distances_per_image.csv"
with open(csv_file_path, mode="w", newline="") as csv_file:
    # Get fieldnames (the keys of the first entry in angles_per_image)
    fieldnames = angles_per_image[0].keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write header and rows
    writer.writeheader()
    writer.writerows(angles_per_image)

print(f"Angles and distances for each image saved to {csv_file_path}")


No keypoints detected in image 1
No keypoints detected in image 2
No keypoints detected in image 113
No keypoints detected in image 224
No keypoints detected in image 335
No keypoints detected in image 446
No keypoints detected in image 557
No keypoints detected in image 668
No keypoints detected in image 1112
No keypoints detected in image 2223
No keypoints detected in image 2989
No keypoints detected in image 2991
No keypoints detected in image 2992
No keypoints detected in image 2993
No keypoints detected in image 2994
No keypoints detected in image 2995
No keypoints detected in image 2996
No keypoints detected in image 2997
No keypoints detected in image 2998
No keypoints detected in image 2999
No keypoints detected in image 3000
No keypoints detected in image 3003
No keypoints detected in image 3004
No keypoints detected in image 3005
No keypoints detected in image 3006
No keypoints detected in image 3007
No keypoints detected in image 3008
No keypoints detected in image 3009
No k